# Supplementary material on Section 4

**We remark that the execution of this script takes several hours, due to the computing time measurement which runs several experiments.**

This code block allows to replicate replicate Section 4.2. Comparison of the methods for computing the cdf of the [manuscript](https://arxiv.org/abs/2303.01129) describing the usage and the design of the GEMAct package.

This vignette is relative to the version 1.2.0 of the GEMAct software.


In [ ]:
pip install gemact==1.2.0

# Sensitivity analysis

In [ ]:
from gemact import Margins, Copula
from gemact.calculators import MCCalculator, AEPCalculator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# local path to save
localpath = ''
explort_flag = False

# array to store outputs
copula1 = 'gaussian'
copula2 = 'clayton'
container = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]

# values where the cdf is evaluated in 2d and 3d
sValues = np.array([[1.25, 1.5, 1.75, 1.95],
                    [2, 2.3, 2.65, 2.85]])
nsim = 10**7

# create Margins
margins = Margins(dist=['uniform', 'uniform'], par=[{'a': 0, 'b': 1}, {'a': 0, 'b': 1}])
margins3 = Margins(dist=['uniform', 'uniform', 'uniform'],
                   par=[{'a': 0, 'b': 1}, {'a': 0, 'b': 1}, {'a': 0, 'b': 1}])

### Gaussian copulas

In [ ]:
# create Copula objects
# correlation values
correlations = np.linspace(-0.95, 0.95, num=19, endpoint=True)
copulas = [
    Copula(dist='gaussian',
           par={'corr': np.array([[1, corr], [corr, 1]])}
           )
    for corr in correlations
    ]

for i in range(len(copulas)):
    print(i)
    cop = copulas[i]
    p_aep = AEPCalculator.cdf(sValues[0, ], n_iter=7, copula=cop, margins=margins)
    container[0][copula1 + ': ' + str(correlations[i])] = p_aep

    simul = MCCalculator.rvs(size=nsim, random_state=55+i, copula=cop, margins=margins).reshape(-1, 1)
    simul = simul < sValues[0, ]
    p_mc = np.mean(simul, axis=0)
    container[1][copula1 + ': ' + str(correlations[i])] = p_mc
        
# export results
if explort_flag == True:
    container[0].to_csv(str(localpath + '\AEP_sens_' + copula1 + '.csv'), sep=",", index=False)
    container[1].to_csv(str(localpath + '\MC_sens_' + copula1 + '.csv'), sep=",", index=False)
    # save as txt
    # np.savetxt(str(localpath + '\AEP_sens_' + copula1 + '.txt'), container[0].values)
    # np.savetxt(str(localpath + '\MC_sens_' + copula1 + '.txt'), container[1].values)


### Clayton Copulas

In [ ]:
# create Copula objects
thetas = np.linspace(0.05, 20, num=19, endpoint=True)
copulas = [
    Copula(dist='clayton',
           par={'par': theta, 'dim': 3}
           )
    for theta in thetas
    ]

for i in range(len(copulas)):
    print(i)
    cop = copulas[i]
    p_aep = AEPCalculator.cdf(sValues[1, ], n_iter=7, copula=cop, margins=margins3)
    container[2][copula2 + ': ' + str(thetas[i])] = p_aep

    simul = MCCalculator.rvs(size=nsim, random_state=55+i, copula=cop, margins=margins3).reshape(-1, 1)
    simul = simul < sValues[1, ]
    p_mc = np.mean(simul, axis=0)
    container[3][copula2 + ': ' + str(thetas[i])] = p_mc
        
# export results
if explort_flag == True:
    container[2].to_csv(str(localpath + '\AEP_sens_' + copula2 + '.csv'), sep=",", index=False)
    container[3].to_csv(str(localpath + '\MC_sens_' + copula2 + '.csv'), sep=",", index=False)
    # save as txt
    # np.savetxt(str(localpath + '\AEP_sens_' + copula2 + '.txt'), container[2].values)
    # np.savetxt(str(localpath + '\MC_sens_' + copula2 + '.txt'), container[3].values)

### Reporting the results

In [ ]:
# plot
coloraep = 'black'
colormc = 'pink'
colorerror = 'red'
colortaberror = 'tab:red'
limiterror = [0, 0.0005]
linestylemc = 'dashed'
linestyleerror = 'dotted'
fontsize_title= 15
fontsize_axis = 15
linewidth_aep = 1
pos_text_y = [[0.25, 0.55, 0.85, 0.96],
              [0.08, 0.35, 0.70, 0.90]]
correlations = np.linspace(-1, 1, num=19, endpoint=True) 

fig1 = plt.figure(figsize=(8, 3))

ax1 = fig1.add_subplot(121)
ax1.set_xlabel(r'$\rho$', fontsize=fontsize_axis)
ax1.set_ylabel('cdf', fontsize=fontsize_axis)
ax1.set_title('Gaussian Copula', multialignment='center', fontsize=fontsize_title)
ax1.tick_params(axis='both', which='major', labelsize=15)
for j in range(sValues.shape[1]):
    if j == 0:
        ax1.plot(correlations, container[0].iloc[j].values,
                 color=coloraep,
                 linewidth=linewidth_aep,
                 label="AEP")
    else:
        ax1.plot(correlations, container[0].iloc[j].values,
                linewidth=linewidth_aep,
                color=coloraep
                )
    plt.text(0.75, pos_text_y[0][j],
             's = ' + str(sValues[0, j]),
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax1.transAxes,
             fontsize='large')

ax3 = ax1.twinx()  
ax3.plot(correlations,
         np.abs(container[0] - container[1]).mean(0).values,
         color=colorerror,
         linestyle=linestyleerror,
         label="|AEP - MC|")
ax3.set_ylabel('|AEP - MC|', color=colortaberror, fontsize=fontsize_axis)
ax3.set_ylim(limiterror)
ax3.tick_params(axis='y', labelcolor=colortaberror, labelsize=15)

ax2 = fig1.add_subplot(122)
ax2.set_xlabel(r'$\theta$', fontsize=fontsize_axis)
ax2.set_ylabel('cdf', fontsize=fontsize_axis)
ax2.set_title('Clayton Copula', multialignment='center', fontsize=fontsize_title)
ax2.tick_params(axis='both', which='major', labelsize=15)
for j in range(sValues.shape[1]):
    if j == 0:
        ax2.plot(thetas, container[2].iloc[j].values,
                color=coloraep,
                linewidth=linewidth_aep,
                )
    else:
        ax2.plot(thetas, container[2].iloc[j].values,
                linewidth=linewidth_aep,
                color=coloraep
                )
    plt.text(0.75, pos_text_y[1][j],
             's = ' + str(sValues[1, j]),
             horizontalalignment='center',
             verticalalignment='center',
             transform = ax2.transAxes,
             fontsize='large')
ax4 = ax2.twinx()  
ax4.plot(
    thetas,
    np.abs(container[2] - container[3]).mean(0).values,
    color=colorerror,
    linestyle=linestyleerror,
    )
ax4.set_ylabel('|AEP - MC|', fontsize=fontsize_axis, color=colortaberror)
ax4.set_ylim(limiterror)
ax4.tick_params(axis='y', labelcolor=colortaberror, labelsize=15)
fig1.savefig(str(localpath + '\lossaggr_sens_copula.eps'))

## Accuracy and speed

**We remark that the execution of this script takes several hours due to the computing time measurement which runs several experiments.**



In [ ]:
from gemact.lossaggregation import Margins, Copula
from gemact.calculators import MCCalculator, AEPCalculator
import numpy as np
import pandas as pd
import timeit as timeit

#### script parameters
# local path to save
localpath = ''
explort_flag = True
dimensions = [2, 3, 4, 5]

# number of AEP iterations and MC simulations.
n_iter = [
    [7, 10, 13, 16],
    [7, 9, 11, 13],
    [4, 5, 6, 7],
    [3, 4, 5, 6]
    ]
n_simul = [10**4, 10**5, 10**6, 10**7]

# timeit parameters. Number of repetitions for computation time.
repetitions = 7 # numer of times the experiment is repeated
number = 100 # number of times the function is executed in a experiment.
# the raw results are expressed in centisecond, i.e. 10**(-2) second.

# values where the cdf is evaluated
sValues = [
    np.array([10.0**x for x in (0, 2, 4, 6)]),
    np.array([10.0**x for x in (0, 2, 4, 6)]),
    np.array([10.0**x for x in (1, 2, 3, 4)]),
    np.array([10.0**x for x in (1, 2, 3, 4)])
]

# containers to store computation accuracy outputs
acc_table = pd.DataFrame()
outputtxt = []


# create Margins and Copula objects
# Example from Arbenz, P., Embrechts, P., & Puccetti, G. (2011).
# The aep algorithm for the fast computation of the distribution of the sum of dependent random variables.
# Bernoulli, 17 (2), 562–591
margins = [
    Margins(
        dist=['genpareto']*2,
        par=[
            {'loc': 0, 'scale': 1/.9, 'c': 1/.9},
            {'loc': 0, 'scale': 1/1.8, 'c': 1/1.8}
            ]),
    Margins(
        dist=['genpareto']*3,
        par=[
            {'loc': 0, 'scale': 1/.9, 'c': 1/.9},
            {'loc': 0, 'scale': 1/1.8, 'c': 1/1.8},
            {'loc': 0, 'scale': 1/2.6, 'c': 1/2.6}
            ]),
    Margins(
        dist=['genpareto']*4,
        par=[
            {'loc': 0, 'scale': 1/.9, 'c': 1/.9},
            {'loc': 0, 'scale': 1/1.8, 'c': 1/1.8},
            {'loc': 0, 'scale': 1/2.6, 'c': 1/2.6},
            {'loc': 0, 'scale': 1/3.3, 'c': 1/3.3}
            ]),
    Margins(
        dist=['genpareto']*5,
        par=[
            {'loc': 0, 'scale': 1/.9, 'c': 1/.9},
            {'loc': 0, 'scale': 1/1.8, 'c': 1/1.8},
            {'loc': 0, 'scale': 1/2.6, 'c': 1/2.6},
            {'loc': 0, 'scale': 1/3.3, 'c': 1/3.3},
            {'loc': 0, 'scale': 1/4, 'c': 1/4}
            ])
    ]

copulas = [
    Copula(dist='clayton', par={'par': 1.2, 'dim': 2}),
    Copula(dist='clayton', par={'par': 0.4, 'dim': 3}),
    Copula(dist='clayton', par={'par': 0.2, 'dim': 4}),
    Copula(dist='clayton', par={'par': 0.3, 'dim': 5})
]

# Monte Carlo cdf function: wrapper for error and computation time
def MCCalculatorCdf(s, size, random_state, copula, margins):
    simul = MCCalculator.rvs(
        size=size,
        random_state=random_state,
        copula=copula,
        margins=margins
        ).reshape(-1, 1)
    simul = (simul <= s)
    prob = np.mean(simul, axis=0)
    return (prob, simul)

# reference value: taken from Arbenz's paper
rifVals = [
    # d0 = 2
    [0.315835041363441, 0.983690398913354, 0.999748719229367, 0.999996018908404],
    # d0 = 3
    [0.190859309689430, 0.983659549676444, 0.999748708770280, 0.999996018515584],
    # d0 = 4
    [0.833447516734442, 0.983412214152579, 0.997950264030106, 0.999742266243751],
    # d0 = 5
    [0.824132635126808, 0.983253494805448, 0.997930730055234, 0.999739803851201]
]
    
for i in range(len(dimensions)):
    dim = dimensions[i]
    print('dim = ' + str(dim))
    acc_table['ref_val_'+str(dim)] = rifVals[i]
    # AEP
    print('AEP')
    for ni in n_iter[i]:
        print('n_iter = ' + str(ni))
        # 1) computation accuracy
        p_aep = AEPCalculator.cdf(sValues[i], n_iter=ni, copula=copulas[i], margins=margins[i])
        err_aep = (rifVals[i] - p_aep) / rifVals[i]
        acc_table['val_aep_'+str(ni)+'_'+str(dim)] = p_aep
        acc_table['err_aep_'+str(ni)+'_'+str(dim)] = err_aep
        "{:.2e}".format(err_aep.astype(str))
        txt = ["{:.2e}".format(x) for x in p_aep]
        
        # 2) computation time
        stmt_aep = "AEPCalculator.core_cdf(" \
                                "x=sValues["+ str(i) + "][0],"\
                                "n_iter=" + str(ni) + ","\
                                "copula=copulas["+ str(i) + "],"\
                                "margins=margins["+ str(i) + "]," \
                                ")"
        out = timeit.repeat(stmt_aep,
                            repeat=repetitions,
                            number=number,
                            globals=locals())
        
        txt.insert(0, str(round(np.min(out) / number, 2)))
        txt.insert(0, '&AEP (' + str(ni) + ' iter.)')
        outputtxt.append('& '.join(txt) + '\\\\')

    # MC
    print('MC')
    random_state = int(10 + dim)
    for ns in n_simul:
        print('n_simul = ' + str(ns))
        # 1) computation accuracy
        p_mc, simul = MCCalculatorCdf(sValues[i], ns, random_state, copula=copulas[i], margins=margins[i])
        err_mc = (rifVals[i] - p_mc) / rifVals[i]
        emc_mc = np.std(simul, axis=0) / np.sqrt(ns)
        acc_table['val_mc_'+str(ns)+'_'+str(dim)] = p_mc
        acc_table['err_mc_'+str(ns)+'_'+str(dim)] = err_mc
        acc_table['emc_mc_'+str(ns)+'_'+str(dim)] = emc_mc
        random_state = (2 * random_state)
        # 2) computation time
        stmt_mc = "MCCalculatorCdf(" \
                    "s=sValues["+ str(i) + "][0],"\
                    "size=" + str(ns) + ","\
                    "random_state=random_state," \
                    "copula=copulas["+ str(i) + "],"\
                    "margins=margins["+ str(i) + "]," \
                    ")"
        out = timeit.repeat(stmt_mc,
                            repeat=repetitions,
                            number=number,
                            globals=locals())
        
        txt = ["{:.2e}".format(x) for x in err_mc]
        txt.insert(0, str(round(np.min(out) / number, 2)))
        txt.insert(0, '&MC (' + str(ns) + ' sim.)')
        outputtxt.append('& '.join(txt) + '\\\\')

# export results
if explort_flag:
    acc_table.to_csv(str(localpath + '\lossaggr_accuracy_vals.csv'), sep=",", index=False)
    pd.DataFrame(data = outputtxt, columns=['text']).to_csv(localpath + 'table_content.txt', index=False)
    print('finish')